<a href="https://www.kaggle.com/hrialan/transfer-learning?scriptVersionId=89755207" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
running_on_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') != 'Localhost'

if running_on_kaggle:
    print('Kaggle environment detected')
    # !nvidia-smi
    data_dir = '/kaggle/input/food41/images/'
else:
    print('Running on localhost')
    data_dir = './dataset/food/images/'
    !pip install --quiet -r requirements.txt
    !python download_data.py

Kaggle environment detected


In [3]:
batch_size = 64

img_height = 224
img_width = 224

In [4]:
image_data_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.2)

In [5]:
train_data = image_data_generator.flow_from_directory(
    data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training')

val_data = image_data_generator.flow_from_directory(
    data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode='categorical',
    subset = 'validation')

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


# ResNet50V2

In [6]:
"""
ResNet50V2 = tf.keras.applications.resnet_v2.ResNet50V2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    classes=101
)

for layer in ResNet50V2.layers:
    layer.trainable = False
"""

"\nResNet50V2 = tf.keras.applications.resnet_v2.ResNet50V2(\n    input_shape=(224, 224, 3),\n    include_top=False,\n    weights='imagenet',\n    pooling='avg',\n    classes=101\n)\n\nfor layer in ResNet50V2.layers:\n    layer.trainable = False\n"

In [7]:
"""
model_ResNet = tf.keras.Sequential([
    ResNet50V2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax')
    ])

model_ResNet.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(lr=.0001), 
    metrics=['accuracy'])
"""

"\nmodel_ResNet = tf.keras.Sequential([\n    ResNet50V2,\n    tf.keras.layers.Flatten(),\n    tf.keras.layers.Dense(101, activation = 'softmax')\n    ])\n\nmodel_ResNet.compile(\n    loss='categorical_crossentropy', \n    optimizer=tf.keras.optimizers.Adam(lr=.0001), \n    metrics=['accuracy'])\n"

In [8]:
"""
epochs=10
history = model_ResNet.fit_generator(
  train_data,
  validation_data=val_data,
  epochs=epochs
)
"""

'\nepochs=10\nhistory = model_ResNet.fit_generator(\n  train_data,\n  validation_data=val_data,\n  epochs=epochs\n)\n'

# InceptionResnetV2

In [9]:
"""
InceptionResNetV2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    classes=101
)

for layer in InceptionResNetV2.layers:
    layer.trainable = False
    
x = InceptionResNetV2.layers[-1].output
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)

output = layers.Dense(101, activation='softmax')(x)
model = models.Model(InceptionResNetV2.input, output)

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=.0001), 
              metrics=['accuracy'])
"""

"\nInceptionResNetV2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(\n    input_shape=(224, 224, 3),\n    include_top=False,\n    weights='imagenet',\n    pooling='avg',\n    classes=101\n)\n\nfor layer in InceptionResNetV2.layers:\n    layer.trainable = False\n    \nx = InceptionResNetV2.layers[-1].output\nx = layers.BatchNormalization()(x)\nx = layers.Dropout(0.2)(x)\nx = layers.Dense(128, activation='relu')(x)\nx = layers.Dropout(0.2)(x)\n\noutput = layers.Dense(101, activation='softmax')(x)\nmodel = models.Model(InceptionResNetV2.input, output)\n\nmodel.compile(loss='categorical_crossentropy', \n              optimizer=tf.keras.optimizers.Adam(lr=.0001), \n              metrics=['accuracy'])\n"

In [10]:
# model.summary()

In [11]:
"""
epochs=10
history = model.fit_generator(
  train_data,
  validation_data=val_data,
  epochs=epochs
)
"""

'\nepochs=10\nhistory = model.fit_generator(\n  train_data,\n  validation_data=val_data,\n  epochs=epochs\n)\n'

# InceptionV3

In [12]:
InceptionV3 = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    classes=101
)

for layer in InceptionV3.layers:
    layer.trainable = False
    
x = InceptionV3.layers[-1].output
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

output = layers.Dense(101, activation='softmax')(x)
model = models.Model(InceptionV3.input, output)

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=.0001), 
              metrics=['accuracy'])

2022-03-10 13:02:50.805299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 13:02:50.877052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 13:02:50.877799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 13:02:50.878963: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

87924736/87910968 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [13]:
epochs=10
history = model.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs
)

2022-03-10 13:02:57.102633: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-03-10 13:03:03.339341: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1263/1263 [==============================] - 2294s 2s/step - loss: 2.9972 - accuracy: 0.3029 - val_loss: 2.3163 - val_accuracy: 0.4317
Epoch 2/10
1263/1263 [==============================] - 1539s 1s/step - loss: 2.4173 - accuracy: 0.4084 - val_loss: 2.1611 - val_accuracy: 0.4592
Epoch 3/10
1263/1263 [==============================] - 1555s 1s/step - loss: 2.2636 - accuracy: 0.4398 - val_loss: 2.0915 - val_accuracy: 0.4788
Epoch 4/10
1263/1263 [==============================] - 1532s 1s/step - loss: 2.1678 - accuracy: 0.4584 - val_loss: 2.0430 - val_accuracy: 0.4876
Epoch 5/10
1263/1263 [==============================] - 1518s 1s/step - loss: 2.1099 - accuracy: 0.4710 - val_loss: 2.0279 - val_accuracy: 0.4958
Epoch 6/10
1263/1263 [==============================] - 1577s 1s/step - loss: 2.0512 - accuracy: 0.4802 - val_loss: 2.0014 - val_accuracy: 0.4954
Epoch 7/10
1263/1263 [==============================] - 1539s 1s/step - loss: 2.0117 - accuracy: 0.4883 - val_loss: 1.9797 - val_accura

In [14]:
model.save('/kaggle/working/model_saved/')

2022-03-10 17:35:25.327973: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
